Linear Regression
The model assumes that the price is a weighted sum of features.
Each feature has a coefficient corresponding to it.
During training, the algorithm selects these coefficients so as to minimize the root-mean-square error between the real price and the predicted one. The advantage of this method is a clear interpretation: the sign and magnitude of the coefficient show how this or that parameter affects the price.

Random Forest Regression
This is a set of multiple decision trees. Each tree is built on a random subsample of objects and features and learns to divide the data by conditions.
When predicting, each tree gives its own price estimate, and the final forecast is the average of all trees.
Due to averaging, the model copes better with noise, captures nonlinear dependencies and interactions of features, and usually makes more accurate predictions than a single tree or a simple linear model.

Russian Edition:

Linear Regression
Модель предполагает, что цена - это взвешенная сумма признаков.
Каждому признаку соответствует коэффициент.
Во время обучения алгоритм подбирает эти коэффициенты так, чтобы минимизировать среднеквадратичную ошибку между реальной ценой и предсказанной. Плюс этого метода — понятная интерпретация: по знаку и величине коэффициента видно, как тот или иной параметр влияет на цену.

Random Forest Regression
Это набор из множества деревьев решений. Каждое дерево строится на случайной подвыборке объектов и признаков и учится делить данные по условиям.
При предсказании каждое дерево выдаёт свою оценку цены, а итоговый прогноз - это среднее по всем деревьям.
За счёт усреднения модель лучше справляется с шумом, улавливает нелинейные зависимости и взаимодействия признаков и обычно даёт более точные прогнозы, чем одиночное дерево или простая линейная модель.

# Prepare of dataset and prepare of pipeline


In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

df = pd.read_csv("computer_prices_all.csv")

y = df["price"]
X = df.drop(columns=["price"])

cat_cols = [
    "device_type", "brand", "model", "os", "form_factor",
    "cpu_brand", "cpu_model", "gpu_brand", "gpu_model",
    "storage_type", "display_type", "resolution",
    "wifi", "bluetooth"
]

num_cols = [c for c in X.columns if c not in cat_cols]

num_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median"))
])

cat_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("encoder", OneHotEncoder(handle_unknown="ignore"))
])

preprocess = ColumnTransformer(
    transformers=[
        ("cat", cat_transformer, cat_cols),
        ("num", num_transformer, num_cols),
    ]
)

In [ ]:

lin_model = Pipeline(steps=[
    ("preprocess", preprocess),
    ("model", LinearRegression())
])

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

lin_model.fit(X_train, y_train)

y_pred = lin_model.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print(f"LR MAE:  {mae:.2f}")
print(f"LR RMSE: {rmse:.2f}")

LR MAE:  164.95
LR RMSE: 234.87


In [ ]:


rf_model = Pipeline(steps=[
    ("preprocess", preprocess),
    ("model", RandomForestRegressor(
        n_estimators=20, # 10
        max_depth=17, # 17, 164,31
        random_state=42,
        n_jobs=-1 # Прошу юзать все ядра, а то оно слишком туго работает
    ))
])

rf_model.fit(X_train, y_train)

y_pred = rf_model.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print(f"RF MAE:  {mae:.2f}")
print(f"RF RMSE: {rmse:.2f}")


RF MAE:  161.79
RF RMSE: 224.25
